<h1>Coding Session #2 - Neuronale Netze</h1>

Diese Datei ist ein Jupyter Notebook. Dieses besteht aus Textblöcken im Markdown Format und ausführbaren Code-Zellen. Diese erkennen Sie an dem kleinen Pfeilsymbol links daneben.

Führen Sie bitte als erstes folgende Zelle aus, um sicherzustellen, dass die benötigten Bibliotheken installiert sind:

In [ ]:
!pip install -r requirements.txt

## Ziel
In dieser Coding Session soll ein neuronales Netz von Grund auf implementiert und trainiert werden. Sie wenden dabei die in der Vorlesung gelernten theoretischen Grundlagen praktisch an und lernen den Einfluss verschiedener Faktoren auf das Trainingsverhalten kennen.

## 1 Neuronales Netz

Ein neuronales Netz besteht aus mehreren Schichten. Jede Schicht (Layer) beinhaltet mindestens ein Neuron.

<img src="images/neural_network1.png" width="300px"></img>

Jeder Layer sagt für einen Input $X$ einen Output $\hat{Y}$ vorher: $\hat{Y}=W\cdot X+b$

In einem neuronalen Netz verarbeitet jeder Layer den Output des davorliegenden Layers (bzw. der erste Layer den Netzwerk Input $X$), wobei $H$ sogenannte Hidden Layer sind.

<img src="images/neural_network.png" width="300px"></img>

Damit ergibt sich die gesamte Netzwerkfunktion als

$H=W_h\cdot X+b_h$

$\hat{Y}=W_{out}\cdot H + b_{out}$

Die Backpropagation erfolgt Layerweise.

## 2 Code
### 2.1 Basisklasse Module

Wir starten mit der Klasse `Module`. Dies ist die Basisklasse, von der alle weiteren Netzwerkkomponenten erben. Sie beinhaltet ein Grundgerüst, welches in allen Aktivierungs- und Verlustfunktionen sowie in allen Netzwerklayern vorhanden sein muss. Dieses besteht aus

`forward`: die Funktion, die während der Forward Propagation im Netzwerk aufgerufen wird, um Vorhersagen zu treffen<br>
`backward`: die Funktion, die während der Backward Propagation aufgerufen wird, um die Parameteranpassungen vorzunehmen

In [ ]:
import numpy as np

# Base Module Class -> all layers and activations will inherit from this
class Module:
    """Base class for all modules (layers, activations, etc.)"""
    def forward(self, input:np.ndarray):
        raise NotImplementedError

    def backward(self, grad_output, eta = None):
        raise NotImplementedError

### 2.2 Netzwerschichten

Kommen wir nun zu den Netzwerkschichten. Wir beschränken uns hier erst einmal auf den `DenseLayer` (oder Fully Connected Layer), also den einfachsten Schichttyp.

Dieser bekommt im Konstruktor die Parameter `input_size` und `output_size` übergeben, womit die Gewichtsmatrix $W$ und der Biasvektor $b$ initialisiert werden.

**Beispiel:** Folgender Layer hat 3 Inputs und 2 Ouputs.

<img src="images/neural_network1.png" width="300px"></img>

Unsere Gewichtsmatrix (`self.weights`) wird mit normalverteilten Zufallswerten dem shape $(input\_size, output\_size)$ initialisiert, sieht folglich so aus:
$$
    W=\begin{bmatrix}
        W_{00} & W_{10} \\
        W_{01} & W_{11} \\
        W_{02} & W_{12}
    \end{bmatrix}
$$

Unser Biasvektor (`self.biases`) wird mit Nullen mit dem shape $(1, output\_size)$ initialisiert und sieht so aus:
$$
    b=\begin{bmatrix}
        b_0 & b_1
    \end{bmatrix}
$$

#### 2.2.1 Die `forward` Funktion (Forward Propagation)

Die übergebenen Inputs $X$ (`inputs`) liegen batchweise vor (um mehrere Inputs gleichzeitig verarbeiten zu können). Folglich hat der Inputvektor einen shape von $(n, input\_size)$ und sieht so aus:
$$
    X=\begin{bmatrix}
        x_0 & x_1 & x_2 \\
        x_0 & x_1 & x_2 \\
        ...   &     &     \\
        x_0 & x_1 & x_2
    \end{bmatrix}
    \begin{matrix}
        batch\,item\,0 \\
        batch\,item\,1 \\
        ... \\
        batch\,item\,n
    \end{matrix}
$$
wobei $n$ die Anzahl der Input Items im Batch ist.

Die shapes können also nicht nach der Form $\hat{Y}=W\cdot X+b$ verarbeitet werden. Wir wenden hier einen kleinen Trick an und rechnen $\hat{Y}=X\cdot W +b$. So können gestackte Inputs (batches) parallel verarbeitet werden, statt mit einer Schleife über jedes $batch\,item$ zu iterieren. Das Prinzip bleibt aber das gleiche und die Ergebnisse sind identisch und die Verarbeitung ist um ein Vielfaches performanter!

Die Outputs $\hat{Y}$ liegen dann in folgender Form vor:
$$
    \hat{Y}=\begin{bmatrix}
        y_0 & y_1 \\
        y_0 & y_1 \\
        ... \\
        y_0 & y_1
    \end{bmatrix}
    \begin{matrix}
        batch\,item\,0 \\
        batch\,item\,1 \\
        ... \\
        batch\,item\,n
    \end{matrix}
$$

In der `forward` Funktion müssen die Inputs mit `self.inputs = inputs.copy()` gespeichert werden. Diese brauchen wir später in der `backward` Funktion, um die Gradienten zu berechnen.

> <span style="color:#00A1E3">**Aufgabe 1 - Forward Propagation**</span>
>
> <img src="images/task_1_1.png" height="260px"></img>
>
> Führen Sie folgende Zelle aus, um die Lösung anzuzeigen:

In [ ]:
import utilities.test as test

test.task_1_1_solution()

#### 2.2.2 Die `backward` Funktion (Backpropagation)

Die Gradienten werden Schichtweise zurückpropagiert, um die Parameter der einzelnen Layer anzupassen. Die `backward` Funktion bekommt den Gradienten der nachfolgenden Schicht $\frac{\partial L}{\partial Y}$ (`grad_output`) und die Lernrate $\eta$ (`eta`) übergeben. Der Gradient hat einen shape von $(batch\_size, output\_size)$ und sieht so aus:
$$
    \frac{\partial L}{\partial Y}=\begin{bmatrix}
        g_0 & g_1 \\
        g_0 & g_1 \\
        \dots \\
        g_o & g_1
    \end{bmatrix}
    \begin{matrix}
        batch\,item\,0 \\
        batch\,item\,1 \\
        ... \\
        batch\,item\,n
    \end{matrix}
$$

Als erstes berechnen wir den Gewichtsgradienten (`grad_weights`), also die partielle Ableitung nach $W$
$$
    \Delta W\approx\frac{\partial L}{\partial W}=X^T\cdot\frac{\partial L}{\partial Y}
$$
> Hinweis: Durch die Anwendung des Skalarproduktes wird die Summe über die Batch Dimension gebildet. Somit hat $\Delta W$ automatisch den gleichen shape wie $W$, da die Batch Dimension entfällt. Somit ist das Parameterupdate für $W$ später problemlos möglich.

Nun werden die Biasgradienten (`grad_biases`) berechnet, also die partielle Ableitung nach $b$
$$
    \Delta b\approx\frac{\partial L}{\partial b}=\sum_{batch}\frac{\partial L}{\partial Y}
$$
Für den Biasgradienten bilden wir also einfach die Summe über alle Batch Items des Output Gradienten (`grad_output`), um hier ebenfalls eine parallele Verarbeitung aller Batch Items zu gewährleisten und die Batch Dimension zu eliminieren.

Anschließend wird noch der Inputgradient (`grad_input`) berechnet. Dies ist der Rückgabewert der Funktion, welcher dann an den vorgelagerten Layer weitergegeben wird:
$$
    \frac{\partial L}{\partial X}=\frac{\partial L}{\partial Y}\cdot W^T
$$

Abschließend erfolgt natürlich noch das Parameter Update:
$$
    W=W-\eta\cdot\Delta W
$$
$$
    b=b-\eta\cdot\Delta b
$$

> <span style="color:#00A1E3">**Aufgabe 2 - Mean Squared Error**</span>
> 
> <img src="images/task_1_2.png" height="260"></img>
>
> Führen Sie folgende Zelle aus, um die Lösung anzuzeigen:

In [ ]:
import utilities.test as test

test.task_1_2_solution()

> <span style="color:#00A1E3">**Aufgabe 3 - Gradienten-Berechnung**</span>
>
> <img src="images/task_1_3.png" height="160"></img>
>
> Führen Sie folgende Zelle aus, um die Lösung anzuzeigen:

In [ ]:
import utilities.test as test

test.task_1_3_solution()

> <span style="color:#00A1E3">**Aufgabe 4 - Dense Layer**</span>
>
> 1. Erstellen Sie eine Klasse `DenseLayer`, welche von `Module` ableitet.
> 2. Erstellen Sie den Konstruktor (`def __init__(self, input_size, out_put_size)`)
>    - Initialisieren Sie die Gewichtsmatrix $W$ (`self.weights`) mit Hilfe der Funktion `np.random.randn`. Übergeben Sie für den shape `input_size` und `output_size`.
>    - Multiplizieren Sie die Gewichtsmatrix mit `0.1`, um nicht mit zu großen Initialgewichten zu starten $\to$ stabileres Training
>    - Initialisieren Sie die Biases $b$ (`self.biases`) mit Null (`np.zeros`) mit einem shape von $(1,\,input\_size)$
> 3. Erstellen Sie die Funktion `forward(self, input)` für die Vorwärtspropagation
>    - Speichern Sie eine Kopie der `inputs` in `self.inputs`, um diese später für die Backpropagation zur Verfügung zu haben (nutzen Sie `np.copy()`)
>    - Geben Sie den Output in der Form $\hat{Y}=X\cdot W+b$ zurück (nutzen Sie für das Skalarprodukt `np.dot()`)
> 4. Erstellen Sie die Funktion `backward(self, grad_output, eta)` für die Backpropagation. `grad_output` ist hier der Gradient der Nachfolgeschicht $\frac{\partial L}{\partial Y}$.
>    - Berechnen Sie die Gewichtsgradienten $\frac{\partial L}{\partial W}$ (`grad_weights`) nach der Formel $\Delta W\approx\frac{\partial{L}}{\partial W}=X^T\cdot\frac{\partial L}{\partial Y}$
>    - Berechnen Sie die Biasgradienten $\frac{\partial L}{\partial b}$ (`grad_biases`) nach der Formel $\Delta b\approx\frac{\partial L}{\partial b}=\sum_{batch}\frac{\partial L}{\partial Y}$. Nutzen Sie für die Summe `np.sum(..., axis=0, keepdims=True)`
>    - Berechnen Sie den Input Gradienten (`grad_input`) $\frac{\partial L}{\partial X}=\frac{\partial L}{\partial Y}\cdot W^T$. Dies ist der Rückgabewert der Funktion, welcher anschließend vom vorgelagerten Layer weiterverarbeitet wird.
>    - Passen Sie nun die Gewichte (`self.weights`) und Biase (`self.biases`) an:
>      - $W=W-\eta\cdot\Delta W$
>      - $b=b-\eta\cdot \Delta b$
>    - Geben Sie den Inputgradienten am Ende der Funktion zurück

In [ ]:
# Layers  


Glückwunsch, Sie haben nun ihren ersten Fully Connected Layer implementiert - die Basis für den weiteren Lernprozess! Und damit ist auch schon der komplizierteste Teil erledigt.
<br><br><br>
**Test der Implementierung**

Mit folgender Zelle können Sie Ihren Code testen:

In [ ]:
from utilities.data import generate_linear_data
from utilities.visualization import plot_data_points

EPOCHS          = 40
LEARNING_RATE   = 0.1
N               = 20

layer = DenseLayer(1, 1)

X, Y = generate_linear_data(m=0.5, n=-1.0, num_samples=N, noise=0.02)

for epoch in range(1, EPOCHS+1):
    # Forward pass
    Y_hat = layer.forward(X)
    
    # Compute Mean Squared Error Loss
    loss = np.mean((Y_hat - Y) ** 2)
    
    # Backward pass
    grad_loss = 2 * (Y_hat - Y) / N
    layer.backward(grad_loss, LEARNING_RATE)

    # Visualization
    if epoch%10 == 0 or epoch == 1 or epoch == EPOCHS:
        plot_data_points(X, Y, Y_hat, title=f"Epoch {epoch}: Model Predictions")    
    print(f"Epoch {epoch}/{EPOCHS}, Loss: {loss:.4f}")

Der Loss sollte nach 40 Epochen $\le 0.0015$ sein und die Vorhersage (Prediction) sollte die Daten (Ground Truth) relativ gut abbilden.

### 2.3 Aktivierungsfunktionen

Aktivierungsfunktionen sind ein wichtiger Bestandteil des maschinellen Lernens. Ohne sie wären Neuronen darauf beschränkt, lineare Beziehungen zwischen Inputs und Outputs zu lernen. Durch die Einführung von Nichtlinearitäten werden neuronale Netze in die Lage versetzt, hochkomplexe Zusammenhänge zu lernen.

<img src="images/activation_function.png" width="500px"></img>

Aktivierungsfunktionen werden den Neuronen nachgeschaltet und transformieren deren linearen Output $\hat{y}$ in Nichtlinearitäten $g(\hat{y})$.

#### 2.3.1 Rectified Linear Unit (ReLU)

_ReLU_ ist die verbreiteteste Aktivierungsfunktion. Eingesetzt wird _ReLU_ vor allem in versteckten Schichten (Hidden Layers) tiefer neuronaler Netze.<br>
<img src="images/ReLU.png" width="500px"></img>

_ReLU_ setzt alle Inputs kleiner $0$ auf $0$ und verhält sich im positiven Bereich linear mit einer Steigung von $1$. Der Gradient von __ReLU__ ist $0$ für alle $z<0$, anderenfalls $1$.

> <span style="color:#00A1E3">**Aufgabe 5 - ReLU**</span>
> 1. Implementieren Sie die Klasse `ReLU`, welche von `Module` ableitet
> 2. Implementieren Sie die Funktion `forward(self, inputs)`
>    - Speichern Sie die `inputs` in `self.inputs`, um diese später für die Backpropagation zur Verfügung zu haben (nutzen Sie `np.copy()`)
>    - Setzen Sie alle Input Werte, welche kleiner $0$ sind, auf $0$ und geben Sie die `inputs` am Ende der Funktion zurück
> 3. Implementieren Sie die Funktion `backward(self, grad_output)`
>    - Kopieren Sie den Output Gradienten in `grad_input`
>    - Setzen Sie für `grad_input` alle Elemente, die `self.inputs <= 0` entsprechen auf $0$
>    - Geben Sie den Input Gradienten am Ende der Funktion zurück

In [ ]:
# Activation Functions 


**Test der Implementierung**

Mit folgender Zelle können Sie Ihren Code testen:

In [ ]:
import utilities.test as test

relu = ReLU()
test.test_ReLU(relu)

#### 2.3.2 Sigmoid

_Sigmoid_ hat einen S-förmigen Verlauf. Die Outputs sind positive Werte $0<g(z)<1$. _Sigmoid_ wird vor allem genutzt, um numerische Netzwerk Outputs $[-\infty; +\infty]$ in Wahrscheinlichkeitswerte $[0; 1]$ umzuwandeln.

<img src="images/sigmoid.png" width="550px"></img>


> <span style="color:#00A1E3">**Aufgabe 6 - Sigmoid**</span>
> 
> 1. Implementieren Sie die Klasse `Sigmoid`, welche von `Module` ableitet
> 2. Implementieren Sie die Funktion `forward(self, inputs)`
>    - Berechnen Sie den Output nach der Formel $g(z)=\frac{1}{1+exp(-z)}$
>    - Speichern Sie die Outputs ($g(z)$) in `self.outputs`, um diese später für die Backpropagation zur Verfügung zu haben
> 3. Implementieren Sie die Funktion `backward(self, grad_output)`
>    - Berechnen Sie den Input Gradienten nach $g'(z)=g(z)\cdot(1-g(z))$ und geben Sie diesen zurück

**Test der Implementierung**

Mit folgender Zelle können Sie Ihren Code testen:

In [ ]:
import utilities.test as test

sigmoid = Sigmoid()

test.test_sigmoid(sigmoid)

#### 2.3.3 Tanh

_Tanh_ hat wie _Sigmoid_ einen S-förmigen Verlauf. Die Outputs sind Werte $-1<g(z)<1$. _Tanh_ kommt u. a. in Hidden Layers flacherer Netze zum Einstz.

<img src="images/tanh.png" width="550px"></img>

> <span style="color:#00A1E3">**Aufgabe 7 - Tanh**</span>
> 
> 1. Implementieren Sie die Klasse `Tanh`, welche von `Module` ableitet
> 2. Implementieren Sie die Funktion `forward(self, inputs)`
>    - Berechnen Sie den Output nach der Formel $g(z)=tanh(z)$
>    - Speichern Sie die Outputs ($g(z)$) in `self.outputs`, um diese später für die Backpropagation zur Verfügung zu haben
> 3. Implementieren Sie die Funktion `backward(self, grad_output)`
>    - Berechnen Sie den Input Gradienten nach $g'(z)=1-tanh^2(z)$ und geben Sie diesen zurück

**Test der Implementierung**

Mit folgender Zelle können Sie Ihren Code testen:

In [ ]:
import utilities.test as test

tanh = Tanh()

test.test_tanh(tanh)

### 2.4 Verlustfunktionen



> <span style="color:#00A1E3">**Aufgabe 8 - MSE Loss**</span>
>
> 1. Implementieren Sie `MSELoss`, welcher von `Module` ableitet.
> 2. Implementieren Sie die Funktion `forward(self, prediction, target)`
>    - Berechnen Sie den MSE Loss nach der Formel $L_{MSE}=\frac{1}{n}\sum_{i=1}^{n}(Y_i-\hat{Y}_i)^2$ und geben Sie diesen zurück
> 3. Implementieren Sie die Funktion `backward(self, prediction, target)`
>    - Berechnen Sie die Input Gradienten nach $\frac{\partial L}{\partial Y}=2\cdot(\hat{Y}-{Y})$ und geben Sie diese zurück.

In [ ]:
# Loss Functions


### 2.5 Neuronales Netz

Im neuronalen Netz befinden sich mehrere Neuronenschichten. Dieses bietet eine Wrapper-Funktionalität, um die Forward- sowie die Backpropagation für alle Layer auszuführen. Dafür wird der Input an die `forward`Funktion des ersten Layers übergeben, dessen Output als Input des darauffolgenden Layers fungiert, usw..

In der Backpropagation wird der Loss-Gradient Layerweise zurückpropagiert. Dafür wird er zuerst an die `backward` Funktion des letzten Layers übergeben. Diese gibt wiederum einen Gradienten zurück, welche an die `backward` Funktion der davorliegenden Schicht weitergegeben wird, usw..

Zwischen den Layern können sich Aktivierungsfunktionen befinden, die nach dem gleichen Prinzip funktionieren.

> <span style="color:#00A1E3">**Aufgabe 9 - Neuronales Netz**</span>
>
> 1. Implementieren Sie `NeuralNetwork`, welches von `Module` ableitet.
> 2. Erstellen Sie den Konstruktor (`def __init__(self, modules:list[Module])`)
>    - `modules` ist hierbei eine Liste von Objekten der Klasse `Module`. Dies können sowohl Layer als auch Aktivierungsfunktionen sein.
>    - Speichern Sie diese in `self.modules`, um später in der `forward` und `backward` Funktion darauf zugreifen zu können
> 3. Erstellen Sie die Funktion `forward(self, input)` für die Vorwärtspropagation
>    - Iterieren Sie über `self.modules` und rufen Sie für jedes Modul die `forward` Methode auf. Übergeben Sie dieser den `input` als Parameter.
>    - Speichern Sie den Rückgabewert der `forward` Funktion wiederum in Input (Sie überschreiben hier den alten Wert)
>    - Geben Sie am Schluss der Funktion `input` zurück. Dies ist unsere finale Modellvorhersage $\hat{Y}$
> 4. Erstellen Sie die Funktion `backward(self, grad_output, eta)` für die Backpropagation. `grad_output` ist hier der Gradient der Lossfunktion $\frac{\partial L}{\partial Y}$.
>    - Da die Gradienten von der letzten Schicht zur ersten zurückpropagiert werden, muss zuerst das letzte Modul, dann das vorletzte, usw. aufgerufen werden.
>    - Iterieren Sie hierfür bitte in invertierter Reihenfolge über `self.modules`, indem Sie Python Funktion `reversed(self.modules)` nutzen
>    - Da sich in `self.modules` sowohl Aktivierungsfunktionen als auch Layer befinden, muss innerhalb der Iterationsschleife folgende Unterscheidung getroffen werden:
>       - Ist das Modul vom Typ `DenseLayer`, so erwartet dessen `backward` Funktion die Parameter `grad_output` und `eta`
>       - Für Aktivierungsfunktionen wird nur der Parameter `grad_output` erwartet
>       - Prüfen Sie deshalb mit `if isinstance(module, DenseLayer)`, ob es sich um einen Layer handelt und rufen Sie die `module.backward` Funktion mit beiden Paremetern auf, anderenfalls nur mit `grad_output`
>    - Speichern Sie den Rückgabewert der `backward` Funktion in `grad_output`. Sie überschreiben hier den alten Wert, um den Gradienten des entsprechenden Moduls an das vorhergehende Modul zu übergebn
>    - Geben Sie am Ende der Funktion `grad_output` zurück.

In [ ]:
# Neural Network Class


### 2.6 Training

Nun testen wir die Implementierung. Dafür werden sinusförmige Trainingsdaten mit Noise generiert. Das Modell soll lernen, diese Daten vorherzusagen.

> <span style="color:#00A1E3">**Aufgabe 10 - Training**</span>
> 1. Erstellen Sie eine Instanz der Klasse `NeuralNetwork`
>   - Für `layers` übergeben Sie eine Liste, 2-4 Neuronenschichten (`DenseLayer`) beinhaltet
>   - Hierbei soll auf jeden `DenseLayer` eine Aktivierungsfunktion folgen (also `ReLU`, `Sigmoid`oder `Tanh`)
>   - Verzichten Sie nach dem letzten Layer auf die Aktivierungsfunktion
>   - Übergeben Sie jedem `DenseLayer` die Anzahl der Inputneuronen sowie die Anzahl der Outputneuronen.
>       - Die erste Neuronenschicht hat lediglich 1 Inputneuron
>       - Die letzte Neuronenschicht hat lediglich 1 Outputneuron
>       - Jede Anzahl der Inputneuronen jeder Folgeschicht muss der Anzahl der Outputneuronen der vorherigen entsprechen
> 2. Experimentieren Sie mit der Modellarchitektur.
> 3. Finden Sie geeignete Werte für die Anzahl der Trainingsepochen (`EPOCHS`), die Lernrate $\eta$ (`LEARNING_RATE`) und die Batchgröße (`BATCH_SIZE`)

In [ ]:
from utilities.data import generate_sinusoidal_data                 # Custom module for generating sinusoidal data
from utilities.visualization import plot_data_points, plot_series   # Custom module for visualizing 2D data

N               :int    = 1000      # Number of data points  
EPOCHS          :int    = None      # Number of training epochs
LEARNING_RATE   :float  = None      # Learning Rate
BATCH_SIZE      :int    = None      # Batch size for training

# Generate sinusoidal data
X, Y = generate_sinusoidal_data(
    num_samples = N,
    noise       = 0.05
)

# plot data points
plot_data_points(X, Y)

model = NeuralNetwork(
    layers = [
        # Your code here
    ]
)
loss_fn = MSELoss()

losses = []

for epoch in range(1, EPOCHS+1):
    loss_epoch = 0.
    indices = np.random.choice(range(N), N, replace=False)
    Y_hat = np.zeros_like(Y)

    for batch in range(0, N, BATCH_SIZE):
        batch_indices = indices[batch:batch + BATCH_SIZE]
        X_batch = X[batch_indices]
        Y_batch = Y[batch_indices]

        # Forward pass
        Y_hat_batch = model.forward(X_batch)

        # Compute loss
        loss = loss_fn.forward(Y_hat_batch, Y_batch)
        loss_epoch += loss

        # Backward pass
        grad_loss = loss_fn.backward(Y_hat_batch, Y_batch)
        model.backward(grad_loss, LEARNING_RATE)

        # store predictions
        Y_hat[batch_indices] = Y_hat_batch
    loss_epoch /= (N / BATCH_SIZE)
    losses.append(loss_epoch)

    if epoch % 5 == 0 or epoch == 1 or epoch == EPOCHS:
        plot_data_points(input=X, target=Y, prediction=Y_hat, title=f"Sinusoidal Data Fitting - Epoch {epoch}")
    print(f"Epoch {epoch}/{EPOCHS}, MSE: {loss_epoch:0.6f}")

plot_series(data=losses, title="Training Loss over Epochs", xlabel="Epochs", ylabel="MSE Loss")
print("Training complete.")

> <span style="color:#00A1E3">**Aufgabe 11 - Experimente**</span>
> 1. Entfernen Sie die Aktivierungsfunktionen zwischen den Layern. Was passiert?
> 2. Ersetzen Sie die `ReLU`Aktivierungen durch `Tanh`und anschließen durch `Sigmoid`. Welchen Effekt hat dies auf das Ergebnis (`MSE` Werte vergleichen)?
> 3. Setzen Sie die `BATCHSIZE=N`, um während des Trainings nicht batchweise, sondern auf dem gesamten Datensatz zu optimieren. Was passiert?